#### Code that fits samples from von Quadt et al. (2014) to a variety of apparent Pb loss functions
##### Accompanyment to "Modeling apparent Pb loss in zircon U-Pb geochronology", submitted to Geochronology
By: Glenn R. Sharman, Department of Geosciences, University of Arkansas

In [ ]:
import convFuncs as convFunc

import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import norm
from scipy.stats import kstest
from astropy.stats import kuiper
from scipy.signal import convolve
from scipy.optimize import minimize

import pandas as pd

import pathlib

import xlsxwriter

import detritalpy.detritalFuncs as dFunc

from KDEpy import FFTKDE

from importlib import reload

import matplotlib
%matplotlib inline
%config InlineBackend.figure_format = 'retina' # For improving matplotlib figure resolution
matplotlib.rcParams['pdf.fonttype'] = 42 # For allowing preservation of fonts upon importing into Adobe Illustrator
matplotlib.rcParams['ps.fonttype'] = 42

In [ ]:
data = pd.read_excel('von_Quadt_input_data.xlsx')

In [ ]:
np.unique(data.Sample_ID)

In [ ]:
n_x = 20001 # Number of x-axis values

##### Choose which sample by running its respective cell below

In [ ]:
# DG026 Granodiorite Ezeris complex
label = 'DG026'
sample_nonCA = 'DG026 (non-CA)'
sample_CA = 'DG026 (CA)'
age = 76.413 # Ma
age_2s_uncert = 0.45 # Myr
xage_1 = 0
xage_2 = age*2
xage = np.linspace(xage_1, xage_2, n_x)
bw = 0.5
xaxis_1 = 65 # For plotting
xaxis_2 = 80 # For plotting

# For final plot
xlim = (65, 90)
xlim_Pb_loss = (-20, 1)
plot_ref_age = True

# Parameters for Pb loss x-axis (%)
x1 = -100 # Note, it is not possible for a U-Pb date to be < -100% from it's true age, as this would result in a negative age
x2 = 100
x = np.linspace(x1, x2, n_x)

In [ ]:
# 059-1 Andesite Borov Dol
label = '059-1'
sample_nonCA = '059-1 (non-CA)'
sample_CA = '059-1 (CA)'

age = 24.57
age_2s_uncert = 0.28

xage_1 = 0
xage_2 = age*2
xage = np.linspace(xage_1, xage_2, n_x)
bw = 0.5
xaxis_1 = 18
xaxis_2 = 32

# For final plot
xlim = (18, 32)
xlim_Pb_loss = (-20, 1)
plot_ref_age = True

# Parameters for Pb loss x-axis (%)
x1 = -100 # Note, it is not possible for a U-Pb date to be < -100% from it's true age, as this would result in a negative age
x2 = 100
x = np.linspace(x1, x2, n_x)

In [ ]:
# 029-5 Andesite Borov Dol
label = '029-5'
sample_nonCA = '029-5 (non-CA)'
sample_CA = '029-5 (CA)'

age = 24.480 # Ma
age_2s_uncert = 0.084 # Myr

xage_1 = 0
xage_2 = age*2
xage = np.linspace(xage_1, xage_2, n_x)
bw = 0.5
xaxis_1 = 18
xaxis_2 = 28

# For final plot
xlim = (18, 32)
xlim_Pb_loss = (-20, 1)
plot_ref_age = True

# Parameters for Pb loss x-axis (%)
x1 = -100 # Note, it is not possible for a U-Pb date to be < -100% from it's true age, as this would result in a negative age
x2 = 100
x = np.linspace(x1, x2, n_x)

In [ ]:
# 284-2 Andesite Borov Dol
label = '248-2'
sample_nonCA = '248-2 (non-CA)'
sample_CA = '248-2 (CA)'

age = 24.422 # Ma
age_2s_uncert = 0.025 # Myr

xage_1 = 0
xage_2 = age*2
xage = np.linspace(xage_1, xage_2, n_x)
bw = 0.5
xaxis_1 = 20 # For plotting
xaxis_2 = 27 # For plotting

# For final plot
xlim = (20, 30)
xlim_Pb_loss = (-20, 1)
plot_ref_age = True

# Parameters for Pb loss x-axis (%)
x1 = -100 # Note, it is not possible for a U-Pb date to be < -100% from it's true age, as this would result in a negative age
x2 = 100
x = np.linspace(x1, x2, n_x)

In [ ]:
# AvQ 244 Granite Trun region Bulgaria
label = 'AvQ 244'
sample_nonCA = 'AvQ244 (non-CA)'
sample_CA = 'AvQ244 (CA)'

age = 333 # Ma
age_2s_uncert = 0.66 # Myr

xage_1 = 0
xage_2 = age*2
xage = np.linspace(xage_1, xage_2, n_x)
bw = 3
xaxis_1 = 260 # For plotting
xaxis_2 = 360 # For plotting

# For final plot
xlim = (270, 390)
xlim_Pb_loss = (-20, 1)
plot_ref_age = True

# Parameters for Pb loss x-axis (%)
x1 = -100 # Note, it is not possible for a U-Pb date to be < -100% from it's true age, as this would result in a negative age
x2 = 100
x = np.linspace(x1, x2, n_x)

##### Then run the code below

In [ ]:
dates_nonCA = np.asarray(data[data['Sample_ID'] == sample_nonCA]['68age'])
errors_nonCA = np.asarray(data[data['Sample_ID'] == sample_nonCA]['68age_err1s'])
dates_CA = np.asarray(data[data['Sample_ID'] == sample_CA]['68age'])
errors_CA = np.asarray(data[data['Sample_ID'] == sample_CA]['68age_err1s'])

In [ ]:
# Filter out old analyses (selected samples only)
if sample_nonCA == 'AvQ244 (non-CA)':
    errors_nonCA = errors_nonCA[dates_nonCA<360]
    dates_nonCA = dates_nonCA[dates_nonCA<360]
    errors_CA = errors_CA[dates_CA<360]
    dates_CA = dates_CA[dates_CA<360]
if sample_nonCA == '029-5 (non-CA)':
    errors_nonCA = errors_nonCA[dates_nonCA<28]
    dates_nonCA = dates_nonCA[dates_nonCA<28]
    errors_CA = errors_CA[dates_CA<28]
    dates_CA = dates_CA[dates_CA<28]
if sample_nonCA == '059-1 (non-CA)':
    errors_nonCA = errors_nonCA[dates_nonCA<28]
    dates_nonCA = dates_nonCA[dates_nonCA<28]
    errors_CA = errors_CA[dates_CA<28]
    dates_CA = dates_CA[dates_CA<28]

In [ ]:
# Plot the non-CA vs CA as a KDE
KDE_nonCA = FFTKDE(bw=bw, kernel='gaussian').fit(dates_nonCA).evaluate(xage)
KDE_nonCA = KDE_nonCA/np.sum(KDE_nonCA)

KDE_CA = FFTKDE(bw=bw, kernel='gaussian').fit(dates_CA).evaluate(xage)
KDE_CA = KDE_CA/np.sum(KDE_CA)

In [ ]:
plt.plot(xage, KDE_nonCA, color='red')
plt.plot(xage, KDE_CA, color='black')
plt.axvline(age, color='black')
plt.xlim(xaxis_1, xaxis_2)
plt.ylim(0,)

In [ ]:
reload(convFunc);

dist_types = ['none','constant','isolated','uniform','gamma','expon','rayleigh','weibull','pareto','halfnorm','lognorm'] # Select which form(s) of Pb loss you want to model

method = 'ss' # 'ss' is sum of squared residuals between ECDF and modeled CDF

omega = 1 # Guess, Myr

pathlib.Path(str(label)).mkdir(parents=True, exist_ok=True) # Recursively creates the directory and does not raise an exception if the directory already exists 

file_name = str(label)+'/'+'model_results_'+label+'.xlsx'

plot_fig = True

workbook = xlsxwriter.Workbook(file_name)

bold_format = workbook.add_format({'bold' : True})

max_offset = (age-np.min(dates_nonCA))/age*-100

# Record model parameters
worksheet = workbook.add_worksheet('Model_parameters')
worksheet.write(0, 0, 'Sample', bold_format)
worksheet.write(1, 0, 'N (non-CA)', bold_format)
worksheet.write(0, 1, label)
worksheet.write(1, 1, len(dates_nonCA))
if dates_CA is not None:
    worksheet.write(2, 0, 'N (CA)', bold_format)
    worksheet.write(2, 1, len(dates_CA))
    worksheet.write(3, 0, 'Misfit function', bold_format)
    worksheet.write(3, 1, method)
else:
    worksheet.write(2, 0, 'Misfit function', bold_format)
    worksheet.write(2, 1, method)
                
c = 0 # Counter variable
worksheet = workbook.add_worksheet('Model_results')
worksheet.write(0, 1, 'fun', bold_format)
worksheet.write(0, 2, 'KS Dmax (f*g)', bold_format)
worksheet.write(0, 3, 'KS p-value (f*g)', bold_format)
worksheet.write(0, 4, 'Kuiper Vmax (f*g)', bold_format)
worksheet.write(0, 5, 'Kuiper p-value (f*g)', bold_format)
if dates_CA is not None:
    worksheet.write(0, 6, 'KS Dmax (f)', bold_format)
    worksheet.write(0, 7, 'KS p-value (f)', bold_format)
    worksheet.write(0, 8, 'Kuiper Vmax (f)', bold_format)
    worksheet.write(0, 9, 'Kuiper p-value (f)', bold_format)
    worksheet.write(0, 10, 'f(t) age', bold_format)
    c += 5
worksheet.write(0, 6+c, 'f(t) 1 s.d.', bold_format)
worksheet.write(0, 7+c, 'g(t) params[0]', bold_format)
worksheet.write(0, 8+c, 'g(t) params[1]', bold_format)
worksheet.write(0, 9+c, 'g(t) params[2]', bold_format)

# First step is to find the best fit for f(t)
params_0 = [age, omega] # Age (Ma), omega (Myr)
bounds = ([age-age_2s_uncert, age+age_2s_uncert], [0,None])
result_f = minimize(convFunc.misfit_norm, params_0, args=(xage, dates_CA, method), bounds=bounds, tol=1e-20)

c = 0 # counter variable
for dist_type in dist_types:
    print('Starting ',dist_type)
    if dist_type == 'none':
        params_0 = [0] # Age (Ma), omega (Myr), and shift in %
        bounds = [(0,0)]
        result = minimize(convFunc.misfit_conv, params_0, args=(dist_type, result_f.x[0], result_f.x[1], x, xage, dates_nonCA, method), 
                      bounds=bounds, tol=1e-20, method='Powell', options={'maxiter' : 1e6, 'disp' : False})

    if dist_type == 'constant':
        params_0 = [-2.0] # Age (Ma), omega (Myr), and shift in %
        bounds = [(max_offset,0)]
        result = minimize(convFunc.misfit_conv, params_0, args=(dist_type, result_f.x[0], result_f.x[1], x, xage, dates_nonCA, method), 
                      bounds=bounds, tol=1e-20, method='Powell', options={'maxiter' : 1e6, 'disp' : False})

    if dist_type == 'isolated':
        params_0 = [-3, 0.8] # Age (Ma), omega (Myr), and shift in %, and proportion of grains with shift (0-1)
        bounds = [(max_offset,0), (0,1)]
        result = minimize(convFunc.misfit_conv, params_0, args=(dist_type, result_f.x[0], result_f.x[1], x, xage, dates_nonCA, method), 
                      bounds=bounds, tol=1e-20, method='Powell', options={'maxiter' : 1e6, 'disp' : False})

    if dist_type == 'expon':
        params_0 = [1.0] # Age (Ma), omega (Myr), and scale
        bounds = [(0,None)]
        result = minimize(convFunc.misfit_conv, params_0, args=(dist_type, result_f.x[0], result_f.x[1], x, xage, dates_nonCA, method), 
                      bounds=bounds, tol=1e-20, method='Powell', options={'maxiter' : 1e6, 'disp' : False})
        
    if dist_type == 'rayleigh':
        params_0 = [1.0] # Age (Ma), omega (Myr), and scale
        bounds = [(0,None)]
        result = minimize(convFunc.misfit_conv, params_0, args=(dist_type, result_f.x[0], result_f.x[1], x, xage, dates_nonCA, method), 
                      bounds=bounds, tol=1e-20, method='Powell', options={'maxiter' : 1e6, 'disp' : False})

    if dist_type == 'halfnorm':
        params_0 = [1.0] # Age (Ma), omega (Myr), and scale
        bounds = [(0,None)]
        result = minimize(convFunc.misfit_conv, params_0, args=(dist_type, result_f.x[0], result_f.x[1], x, xage, dates_nonCA, method), 
                      bounds=bounds, tol=1e-20, method='Powell', options={'maxiter' : 1e6, 'disp' : False})

    if dist_type == 'lognorm':
        params_0 = [1.0, 1.0] # Age (Ma), omega (Myr), scale, and shape
        bounds = [(0,None), (0,None)]
        result = minimize(convFunc.misfit_conv, params_0, args=(dist_type, result_f.x[0], result_f.x[1], x, xage, dates_nonCA, method), 
                      bounds=bounds, tol=1e-20, method='Powell', options={'maxiter' : 1e6, 'disp' : False})
        
    if dist_type == 'weibull':
        params_0 = [1.0, 2.0] # Age (Ma), omega (Myr), scale, and shape
        bounds = [(0,None),(0,None)]
        result = minimize(convFunc.misfit_conv, params_0, args=(dist_type, result_f.x[0], result_f.x[1], x, xage, dates_nonCA, method), 
                      bounds=bounds, tol=1e-20, method='Powell', options={'maxiter' : 1e6, 'disp' : False})

    if dist_type == 'gamma':
        params_0 = [0.5, 1.0] # Age (Ma), omega (Myr), scale, and shape
        bounds = [(0,None),(0,None)]
        result = minimize(convFunc.misfit_conv, params_0, args=(dist_type, result_f.x[0], result_f.x[1], x, xage, dates_nonCA, method), 
                      bounds=bounds, tol=1e-20, method='Powell', options={'maxiter' : 1e6, 'disp' : False})

    if dist_type == 'uniform':
        params_0 = [1.0, 1.0] # Age (Ma), omega (Myr), u_min, delta_u
        bounds = [(0,None),(0,None)]
        result = minimize(convFunc.misfit_conv, params_0, args=(dist_type, result_f.x[0], result_f.x[1], x, xage, dates_nonCA, method), 
                      bounds=bounds, tol=1e-20, method='Powell', options={'maxiter' : 1e6, 'disp' : False})

    if dist_type == 'pareto':
        params_0 = [1] # Age (Ma), omega (Myr), shape
        bounds = [(0,None)]
        result = minimize(convFunc.misfit_conv, params_0, args=(dist_type, result_f.x[0], result_f.x[1], x, xage, dates_nonCA, method), 
                      bounds=bounds, tol=1e-20, method='Powell', options={'maxiter' : 1e6, 'disp' : False})
    
    Pb_loss_pct_pdf = convFunc.Pb_loss_fun(params=result.x, dist_type=dist_type, x=x)
    
    # First model the Gaussian that best approximates the CA U-Pb dates
    rv_norm_Ma = norm(loc=result_f.x[0], scale=result_f.x[1])
    norm_Ma_pdf = rv_norm_Ma.pdf(xage)
    norm_Ma_pdf = norm_Ma_pdf/np.sum(norm_Ma_pdf)

    conv_Ma_pdf = convolve(Pb_loss_pct_pdf, norm_Ma_pdf, mode='same')

    ks_results = kstest(rvs=dates_nonCA, cdf=convFunc.cdf_fun(xage, conv_Ma_pdf))
    kuiper_results = kuiper(data=dates_nonCA, cdf=convFunc.cdf_fun(xage, conv_Ma_pdf))
    
    if dates_CA is not None:
        ks_results_f = kstest(rvs=dates_CA, cdf=convFunc.cdf_fun(xage, norm_Ma_pdf))
        kuiper_results_f = kuiper(data=dates_CA, cdf=convFunc.cdf_fun(xage, norm_Ma_pdf))
    
    d = 0 # Counter variable
    worksheet.write(c+1, 0, dist_type, bold_format)
    worksheet.write(c+1, 1, result.fun)
    worksheet.write(c+1, 2, ks_results[0])
    worksheet.write(c+1, 3, ks_results[1])
    worksheet.write(c+1, 4, kuiper_results[0])
    worksheet.write(c+1, 5, kuiper_results[1])
    if dates_CA is not None:
        worksheet.write(c+1, 6, ks_results_f[0])
        worksheet.write(c+1, 7, ks_results_f[1])
        worksheet.write(c+1, 8, kuiper_results_f[0])
        worksheet.write(c+1, 9, kuiper_results_f[1])
        d += 5
    worksheet.write(c+1, 10, result_f.x[0])
    worksheet.write(c+1, 11, result_f.x[1])
    for i in range(len(result.x)):
        worksheet.write(c+1, 7+d+i, result.x[i])
       
    print('---{}: '.format(method), np.round(result.fun,6))
    
    for i in range(len(result.x)-2):
        print('---g(t) params[{}]'.format(i),np.round(result.x[i+2],2))
    c+=1
    
    if plot_fig:
        fig = convFunc.plot_Pb_loss_model_approach_1(params_norm = [result_f.x[0], result_f.x[1]], params_Pb_loss=result.x,
                                                     fit=result.fun, dates_input=dates_nonCA, errors_1s_input=errors_nonCA, 
                                xage=xage, x=x, xlim=xlim, xlim_Pb_loss=xlim_Pb_loss, dist_type=dist_type,
                                plot_ref_age=plot_ref_age, ref_age=age, ref_age_2s_uncert=age_2s_uncert, dates_input_CA=dates_CA,
                                                    errors_1s_input_CA=errors_CA);
        fig.savefig(str(label)+'/'+'fig_'+str(dist_type)+'.pdf')
    
workbook.close()